In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.3 MB/s eta 0:00:00


In [ ]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 124.2 MB/s eta 0:00:00


In [ ]:
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Load environment variables from .env file
load_dotenv("gurobi.lic")

# Read values from the environment
access_id = os.getenv("WLSACCESSID")
secret = os.getenv("WLSSECRET")
license_id = int(os.getenv("LICENSEID"))

# Set up the Gurobi environment
env = gp.Env(empty=True)
env.setParam("WLSACCESSID", access_id)
env.setParam("WLSSECRET", secret)
env.setParam("LICENSEID", license_id)
env.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2675272
Academic license 2675272 - for non-commercial use only - registered to a0___@itesm.mx


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=2675272>

In [ ]:
import numpy as np
import pulp
import pandas as pd

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23


In [ ]:
from time import thread_time_ns
# Usar CBC con ratioGap del 10% (como optcr = 0.1)
#solver = pulp.PULP_CBC_CMD(msg=True, options=['gapRel=0.1'],timeLimit=60,threads=8)
env.setParam('MIPGap', 0.36)

solver = pulp.GUROBI(
    msg=True,threads=32   ,env=env         # Muestra el log del solver
          # Usar hasta 16 hilos (tienes 300 GB de RAM
)

Set parameter MIPGap to value 0.36


In [ ]:
# --- Load Data from CSVs ---
try:
    distance_times_df = pd.read_csv('distance_times.csv', index_col=0)
    plantas_df = pd.read_csv('Plantas.csv', index_col=0)
    plantas_vivero_df = pd.read_csv('Plantas_vivero.csv')
    plantas_vol_df = pd.read_csv('Plantas_vol.csv')
    # Try loading Plantas_supervivencia.csv with 'Plant Type' as index_col
    plantas_supervivencia_df = pd.read_csv('Plantas_supervivencia.csv', index_col='Plant Type')
except KeyError:
    # If 'Plant Type' is not found, try 'Plant_Type' or another common variation
    print("Column 'Plant Type' not found in Plantas_supervivencia.csv. Trying 'Plant_Type'...")
    plantas_supervivencia_df = pd.read_csv('Plantas_supervivencia.csv', index_col='Plant_Type')
except Exception as e:
    print(f"An error occurred while loading CSV files: {e}")
    print("Please ensure all CSV files are correctly named and formatted.")
    exit() # Exit if essential files can't be loaded

In [ ]:
max_viajes_por_dia = 100  # se debe verificar este valor
viajes_por_dia = range(1, max_viajes_por_dia + 1)
distance_times_df = pd.read_csv('distance_times.csv', index_col=0)

# --- Constantes ---
EQUIVALENTE_PLANTAS_POR_HA = 0.0069
MIN_HA_POR_DIA_DIA_LABORAL = 5
MIN_HA_POR_DIA_SABADO = 0
HORAS_LABOR_DIA_LABORAL = 6
HORAS_LABOR_SABADO = 3
COSTO_PLANTACION_POR_PLANTA = 20
COSTO_CAMION_VIVERO = 4500
CAPACIDAD_CAMION_VIVERO = 8000

STACK_CAMIONETA=2# CHECAR ESTO
STACK_TRATADO=5

COSTO_DIA_ACTIVO = 10000      # ← ajústalo a tu gusto


PROMEDIO_TRATADO_PLANTAS= 40 # ajustar para cada planta despues

TIEMPO_ANTICIPACION_PEDIDO_DIAS = 0

CAPACIDAD_ALMACEN_M2 = 400
DIMENSION_BASE_PLANTA_CM = 25
AREA_INSTALACION_TRATAMIENTO_M2 = 100
DIM_CAMION_CM = (199, 127)
TIEMPO_CARGA_DESCARGA_CAMION_MIN = 30
TIEMPO_TRATAMIENTO_20_MIN = 20
TIEMPO_TRATAMIENTO_60_MIN = 60
DIA_INICIO_PLANTACION_DESDE_LLEGADA = 3
DIA_FIN_PLANTACION_DESDE_LLEGADA = 7
RENDIMIENTO_LITRO_CAMIONETA = 10  # ejemplo km por litro
PRECIO_GASOLINA = 22  # ejemplo pesos por litro

MIN_PLANTAS_POR_TIPO = {
    'E1': 1178,
    'E2': 8508,
    'E3': 1223,
    'E4': 1217,
    'E5': 2427,
    'E6': 2430,
    'E7': 2426,
    'E8': 2435,
    'E9': 3643,
    'E10': 1202
}

M_BIG = sum(MIN_PLANTAS_POR_TIPO.values())   # o cualquier cota > plantas máximas/día


# Constantes derivadas
AREA_PLANTA_M2 = (DIMENSION_BASE_PLANTA_CM / 100) ** 2
PLANTAS_POR_M2_ALMACEN = CAPACIDAD_ALMACEN_M2 / AREA_PLANTA_M2
PLANTAS_POR_M2_TRATAMIENTO = AREA_INSTALACION_TRATAMIENTO_M2 / AREA_PLANTA_M2

# CAPACIDAD_CAMION_ENTREGA_PLANTAS = int(DIM_CAMION_CM[0] / DIMENSION_BASE_PLANTA_CM) * \
#                                  int(DIM_CAMION_CM[1] / DIMENSION_BASE_PLANTA_CM)

CAPACIDAD_CAMION_ENTREGA_PLANTAS = 80

PLANTAS_POR_HORA_TRABAJO = 15

# Horizonte temporal para la planificación
cant_dias = 210

dias = range(1, cant_dias + 1)

DIAS_SEMANA = {
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday',
    5: 'Saturday', 6: 'Sunday'
}
DIAS_LABORABLES = [d for d, nombre in DIAS_SEMANA.items() if nombre != 'Sunday']


# --- Procesamiento de datos cargados para entradas del modelo ---

# CORRECCIÓN: Cambiar índices y columnas de distance_times_df para tener prefijo 'P'
distance_times_df.index = ['P' + str(i) for i in distance_times_df.index]
distance_times_df.columns = ['P' + str(c) for c in distance_times_df.columns]

# polígonos y polígono de almacenamiento
poligonos = distance_times_df.index.tolist()
POLIGONO_ALMACENAMIENTO = 'P18'

# viveros
viveros = ['V1', 'V2', 'V3', 'V4']

# especies (tipos de planta), excluyendo columnas no relacionadas
especies = [col for col in plantas_df.columns if col not in ['Hectárea', '#_plantas']]

# matriz de distancias (copiada del DataFrame modificado)

matriz_distancias = distance_times_df.copy()

# requerimientos de plantas por polígono
requerimientos_plantas_poligono = plantas_df.T.to_dict('index')

# costos de viveros
COSTO_ALTO = 1000  # costo alto artificial para desalentar selección

costos_viveros = {vivero: {} for vivero in viveros}

for index, row in plantas_vivero_df.iterrows():
    especie = row['Especie']
    for vivero in viveros:
        costo = row[vivero]
        costos_viveros[vivero][especie] = costo if costo > 0 else COSTO_ALTO

# tiempos de tratamiento por planta según columna 'Nopal'
tiempo_tratamiento_planta = {}
for _, row in plantas_vol_df.iterrows():
    especie = row['Especie']
    valor_nopal = row['Nopal']
    if valor_nopal == 0:
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_20_MIN
    elif valor_nopal == 1:
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_60_MIN
    else:
        print(f"Advertencia: Valor inesperado 'Nopal' para {especie}: {valor_nopal}. Usando 20 minutos por defecto.")
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_20_MIN

# matriz de tasa de supervivencia
nuevas_columnas_supervivencia = {}
for col in plantas_supervivencia_df.columns:
    if isinstance(col, str) and 'Día' in col:
        try:
            num_dia = int(col.replace('Día ', ''))
            nuevas_columnas_supervivencia[col] = num_dia
        except ValueError:
            nuevas_columnas_supervivencia[col] = col
    else:
        nuevas_columnas_supervivencia[col] = col

plantas_supervivencia_df = plantas_supervivencia_df.rename(columns=nuevas_columnas_supervivencia)

def intentar_int(x):
    try:
        return int(x)
    except (ValueError, TypeError):
        return x

plantas_supervivencia_df.columns = [intentar_int(col) for col in plantas_supervivencia_df.columns]
matriz_tasa_supervivencia = plantas_supervivencia_df.T.to_dict('index')

# asegurar tasa de supervivencia 0 si se planta fuera de ventana 3-7 días desde llegada
for d in range(1, cant_dias + 1):
    for especie in especies:
        tasa_actual = matriz_tasa_supervivencia.get(d, {}).get(especie, 0.0)

        if d < DIA_INICIO_PLANTACION_DESDE_LLEGADA or d > DIA_FIN_PLANTACION_DESDE_LLEGADA:
            matriz_tasa_supervivencia.setdefault(d, {})[especie] = 0.0
        else:
            matriz_tasa_supervivencia.setdefault(d, {})[especie] = tasa_actual


In [ ]:
# --- Optimization Model Structure (PuLP) ---
# Create the problem
prob = pulp.LpProblem("Reforestation_Optimization", pulp.LpMinimize)

In [ ]:
# --- Optimization Model Structure (PuLP) ---
# Create the problem
prob = pulp.LpProblem("Reforestation_Optimization", pulp.LpMinimize)

# --- Decision Variables ---

#   1. Nùmero de plantas e del vivero v que se pidieron el dia d

x = pulp.LpVariable.dicts(
    "NumPlantas",
    ((e, v, d) for e in especies for v in viveros for d in dias),
    lowBound=0,
    cat='Integer'
)

#   2. Número de plantas e en el almacen en el dia d

a = pulp.LpVariable.dicts(
    "PlantasAlmacen",
    ((e, d) for e in especies for d in dias),
    lowBound=0,
    cat='Integer'
)

#   3. Numero de plantas e que se tratan en dia d

t = pulp.LpVariable.dicts(
    "PlantasTratadas",
    ((e, d) for e in especies for d in dias),
    lowBound=0,
    cat='Integer'
)

#   4. Numero de plantas e que se sembraron en el poligono p en el dia d que llegaron el dia l

pl = pulp.LpVariable.dicts(
    "PlantasSembradas",
    ((e, p, d, l) for e in especies for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7),
    lowBound=0,
    cat='Integer'
)

#   5. Numero de viajes al poligono p en el dia d

num_v = pulp.LpVariable.dicts(
    "ViajesPoligono",
    ((p, d) for p in poligonos for d in dias),
    lowBound=0,
    cat='Integer'
)

#   6. Variable binaria que establece si ese dia se va al poligono p

vi = pulp.LpVariable.dicts(
    "VisitaPoligono",
    ((p, d) for p in poligonos for d in dias),
    cat='Binary'
)

#   7. Numero de plantas e que se llevaron al poligono p en el dia d en el viaje v

c = pulp.LpVariable.dicts(
    "PlantasTransportadasViaje",
    ((e, p, d, viaje) for e in especies for p in poligonos for d in dias for viaje in viajes_por_dia),
    lowBound=0,
    cat='Integer'
)

#   8. Numero de veces que se pidio en el dia d al vivero v
pi = pulp.LpVariable.dicts(
    "PedidosVivero",
    ((v, d) for v in viveros for d in dias),
    lowBound=0,
    cat='Integer'
)

di = pulp.LpVariable.dicts("DiaActivo", dias, cat='Binary')


pedido_activo = pulp.LpVariable.dicts(
    "PedidoActivo",
    ((v, d) for v in viveros for d in dias),
    cat='Binary'
)


In [ ]:
distancia_almacen_poligono = {}
for p in poligonos:
    distancia_almacen_poligono[p] = matriz_distancias.loc['P18', p]

In [ ]:
# --- Objective Function ---

prob += (
    # Costo por pedir plantas al vivero (pi * costo fijo por pedido)
    pulp.lpSum(pi[(v, d)] * COSTO_CAMION_VIVERO for v in viveros for d in dias)
    +
    # Costo por plantas pedidas (sumatoria x * costo planta)
    pulp.lpSum(x[(e, v, d)] * costos_viveros[v][e] for e in especies for v in viveros for d in dias)
    +
    # Costo por plantar plantas (cuádruple sumatoria pl * costo de plantar)
    pulp.lpSum(pl[(e, p, d, l)] * COSTO_PLANTACION_POR_PLANTA for e in especies for p in poligonos for d in dias for l in dias if (e, p, d, l) in pl)
    +
    # Costo gasolina: (distancia / rendimiento) * precio * número de viajes v(p,d)
    pulp.lpSum(
        (distancia_almacen_poligono[p] / RENDIMIENTO_LITRO_CAMIONETA) * PRECIO_GASOLINA * num_v[(p, d)]
        for p in poligonos for d in dias
    )
    + COSTO_DIA_ACTIVO * pulp.lpSum(di[d] for d in dias)
), "CostoTotal"

In [ ]:
# Restricciones

# 1. Pide al menos lo minimo para sembrar las hectareas.
for e in especies:
    prob += pulp.lpSum(pl[(e, p, d, l)]
                       for p in poligonos
                       for d in dias
                       for l in dias
                       if (e, p, d, l) in pl and 3 <= (d - l) <= 7) >= MIN_PLANTAS_POR_TIPO[e], f"Minimo_Plantas_{e}"


# 2. Lo que plantas no debe ser mayor a lo que tratas

for e in especies:
    for d in dias:
        prob += t[(e, d)] >= pulp.lpSum(
            pl[(e, p, d, l)]
            for p in poligonos
            for l in dias
            if (e, p, d, l) in pl and 3 <= (d - l) <= 7
        ), f"Tratamiento_Suficiente_{e}_{d}"

#3. No puedes tratar mas de lo que tienes en almacen

for e in especies:
    for d in dias:
        prob += a[e, d] >= a[e, d], f"Tratado_no_excede_almacen_{e}_{d}"

#4. Flujo del almacen

for e in especies:
    for d in dias:
        if d == 1:
            prob += a[e, d] == 0, f"Inicial_almacen_{e}" # Dia cero del almacen , se inicializa en 0
        else:
            entradas = pulp.lpSum(
                x[e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS]
                for v in viveros
                if (e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS) in x
            )
            salidas = pulp.lpSum(
                pl[e, p, d, l]
                for p in poligonos
                for l in dias
                if (e, p, d, l) in pl
            )
            prob += a[e, d] == a[e, d - 1] + entradas - salidas, f"Flujo_almacen_{e}_{d}"

#5. Volumen

# Usar esta cuando se tenga un volumen diferente por cada tipo de planta
#for d in dias:
#    prob += (
#        pulp.lpSum(a[e, d] * AREA_PLANTA_M2[e] for e in especies) <= PLANTAS_POR_M2_ALMACEN,
#        f"Restriccion_Volumen_Almacen_dia_{d}"
#    )

for d in dias:
    prob += (
        pulp.lpSum(a[e, d] * AREA_PLANTA_M2) <= PLANTAS_POR_M2_ALMACEN,
        f"Restriccion_Volumen_Almacen_dia_{d}"
    )

for d in dias:
    for p in poligonos:
        for v in viajes_por_dia:
            prob += (pulp.lpSum(c[e,p,d,v] * AREA_PLANTA_M2* STACK_CAMIONETA) <= (199*127)*STACK_CAMIONETA,f"Restriccion_Volumen_Camioneta_{p}_{d}_{v}")

for d in dias:
    prob += (
        pulp.lpSum(t[e, d] * AREA_PLANTA_M2* STACK_TRATADO) <= AREA_INSTALACION_TRATAMIENTO_M2*STACK_TRATADO,f"Restriccion_Volumen_Tratado_{e}_{d}_")


# No pueden aparecer plantas mágicas

for e in especies:
    total_plantadas = pulp.lpSum(pl[e, p, d, l] for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7)
    total_pedidas = pulp.lpSum(x[e, v, d] for v in viveros for d in dias)

    prob += (
        total_plantadas <= total_pedidas,
        f"Conservacion_Plantas_{e}"
    )

# Tiempos tratados

for d in dias:
    # Determinar si el día es sábado (5) o un día entre semana (0–4)
    dia_semana = (d - 1) % 7  # Asumiendo día 1 es lunes
    horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
    prob += (
            t[e, d] * PROMEDIO_TRATADO_PLANTAS <= horas_laborales * 60,
            f"TiempoTratamiento_{e}_{d}"
        )
# Tiempos viajes
for p in poligonos:
    if p == "P18":
        continue  # No se hacen viajes del almacén al almacén

    for d in dias:
        # Determinar jornada laboral
        day_of_week = (d - 1) % 7
        jornada = HORAS_LABOR_SABADO if day_of_week == 5 else HORAS_LABOR_DIA_LABORAL

        # Obtener tiempo de viaje ida y vuelta (en minutos)
        tiempo_viaje = matriz_distancias.loc["P18", p] + TIEMPO_CARGA_DESCARGA_CAMION_MIN

        # Restricción: tiempo total de viajes al polígono p en día d no debe exceder jornada
        prob += pulp.lpSum(num_v[p, d] * tiempo_viaje) <= jornada, f"Tiempo_Viajes_{p}_{d}"

# Tiempos plantado

for e in especies:
    for p in poligonos:
        for d in dias:
            dia_semana = (d - 1) % 7  # Día de la semana (0 = lunes, ..., 6 = domingo)

            if dia_semana == 6:
                # No se planta en domingo
                prob += (
                    pulp.lpSum(pl[e, p, d, l] for l in dias if (e, p, d, l) in pl) == 0,
                    f"ProhibidoPlantado_Domingo_{e}_{p}_{d}"
                )
            else:
                horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
                minutos_disponibles = horas_laborales * 60

                prob += (
                    pulp.lpSum(
                        pl[e, p, d, l] * 60/PLANTAS_POR_HORA_TRABAJO
                        for l in dias if (e, p, d, l) in pl
                    ) <= minutos_disponibles,
                    f"TiempoPlantado_{e}_{p}_{d}"
                )

# 8. Flujo de las plantas es menor a las que se transportadan a poligonos

for d in dias:
    for p in poligonos:
        for e in especies:
            plantadas = pulp.lpSum(pl[e, p, d, l] for l in dias if 3 <= (d - l) <= 7)
            transportadas = pulp.lpSum(c[e, p, d, v] for v in viajes_por_dia)

            prob += (
                plantadas <= transportadas,
                f"Flujo_Plantas_{e}_{p}_{d}"
            )

# Flujo plantas a poligonos ese dia menor o igual a las tratadas

for d in dias:
    for e in especies:
        transportadas_dia = pulp.lpSum(c[e, p, d, v] for p in poligonos for v in viajes_por_dia)
        tratadas_dia = pulp.lpSum(t[e, d])

        prob += (
            transportadas_dia <= tratadas_dia,
            f"Flujo_Transporte_Tratamiento_{e}_{d}"
        )

# Restriccion pedido a viveros

for v in viveros:
    for d in dias:
        prob += (
            pulp.lpSum(x[e, v, d] for e in especies if (e, v, d) in x) <= CAPACIDAD_CAMION_VIVERO * pi[v, d],
            f"CapacidadVivero_{v}_{d}"
        )



# Plantadas debe ser menor a las ordenadas hasta ese día
for e in especies:
    for d in dias:
        for l in dias:
            if 3 <= (d - l) <= 7:
                plantadas_limite = pulp.lpSum(pl[e, p, d, l] for p in poligonos)
                ordenadas_limite = pulp.lpSum(x[e, v, l] for v in viveros)

                prob += (
                    plantadas_limite <= ordenadas_limite,
                    f"Balance_Plantadas_Ordenadas_{e}_{d}_{l}"
                )


# Plantar al menos num plants por dias  # no funciona hace que se plante mas de lo que debe
# for p in poligonos:
#     for d in dias:
#         dia_semana = (d - 1) % 7  # 0 = lunes, ..., 6 = domingo

#         if dia_semana == 6:
#             minimo_plantas = 0  # Domingo
#         elif dia_semana == 5:
#             minimo_plantas = 40  # Sábado
#         else:
#             minimo_plantas = 90  # Lunes a viernes

#         prob += (
#             pulp.lpSum(
#                 pl[e, p, d, l]
#                 for e in especies
#                 for l in dias
#                 if (e, p, d, l) in pl and 3 <= (d - l) <= 7
#             ) >= minimo_plantas,
#             f"MinimoPlantado_{p}_{d}"
#         )

for d in dias:
    # Plantas sembradas en el día d (todas las especies, polígonos y llegadas válidas)
    plantadas_d = pulp.lpSum(
        pl[e, p, d, l]
        for e in especies
        for p in poligonos
        for l in dias
        if (e, p, d, l) in pl  # la variable existe
    )

    # Enlace Big-M
    prob += plantadas_d <= M_BIG * di[d],          f"DiaActivo_SUP_{d}"
    prob += plantadas_d >= 10*di[d],                  f"DiaActivo_INF_{d}"

MINIMO_PLANTAS_POR_PEDIDO = 4000  # o el valor que decidas

for v in viveros:
    for d in dias:
        total_pedido = pulp.lpSum(
            x[e, v, d] for e in especies if (e, v, d) in x
        )

        # Restricción superior (máximo del camión)
        prob += total_pedido <= CAPACIDAD_CAMION_VIVERO * pedido_activo[v, d], f"CapacidadMax_{v}_{d}"

        # Restricción inferior (mínimo por pedido)
        prob += total_pedido >= MINIMO_PLANTAS_POR_PEDIDO * pedido_activo[v, d], f"CapacidadMin_{v}_{d}"

        # Enlace con la variable de conteo de pedidos (pi)
        prob += pi[v, d] >= pedido_activo[v, d], f"LinkPedidoActPi_{v}_{d}"




In [ ]:
viajes_por_dia

range(1, 101)

In [ ]:
solver

In [ ]:
prob.solve(solver)

Set parameter Threads to value 32
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  0.36
Threads  32

Academic license 2675272 - for non-commercial use only - registered to a0___@itesm.mx
Optimize a model with 810580 rows, 6848540 columns and 16633650 nonzeros
Model fingerprint: 0xd98d776f
Variable types: 0 continuous, 6848540 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-02, 3e+04]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 5e+04]
Presolve removed 741060 rows and 6015952 columns
Presolve time: 3.52s
Presolved: 69520 rows, 832588 columns, 3110857 nonzeros
Variable types: 0 continuous, 832588 integer (1017 binary)
Performing another presolve...
Presolve removed 0 rows and 543213 columns (preso

1

In [ ]:
from pulp import LpStatus, value

# 1. Status of the solution
print(f"Status: {LpStatus[prob.status]}")

# 2. Objective value
print(f"Objective value (Costo total): {value(prob.objective):,.2f}")

Status: Optimal
Objective value (Costo total): 4,368,513.50


In [ ]:
def extraer_resultados_variable(variable_dict, nombre_columnas=None, nombre_valor="Valor"):
    """
    Convierte un diccionario de variables de PuLP en un DataFrame con valores mayores a cero.

    Parámetros:
        variable_dict (dict): Diccionario de variables de PuLP.
        nombre_columnas (list): Lista opcional de nombres para las columnas de los índices.
        nombre_valor (str): Nombre de la columna para el valor de la variable.

    Retorna:
        pd.DataFrame: DataFrame con los valores positivos de las variables.
    """
    resultados = []

    for indices, variable in variable_dict.items():
        valor = variable.varValue
        if valor is not None and valor > 0:
            if isinstance(indices, tuple):
                fila = dict(zip(nombre_columnas or [f"Index_{i}" for i in range(len(indices))], indices))
            else:
                fila = {nombre_columnas[0] if nombre_columnas else "Index_0": indices}
            fila[nombre_valor] = int(valor) if valor.is_integer() else valor
            resultados.append(fila)

    return pd.DataFrame(resultados)



In [ ]:
dia_act = extraer_resultados_variable(di, nombre_columnas=["Dia"], nombre_valor="Dia act")
dia_act

,Dia,Dia act
0,9,1
1,11,1
2,15,1
3,16,1
4,17,1
...,...,...
129,204,1
130,205,1
131,206,1
132,207,1


In [ ]:
dia_act["Dia act"].sum()

np.int64(134)

In [ ]:
df_t = extraer_resultados_variable(t, nombre_columnas=["Especie", "Día"], nombre_valor="Cantidad_Tratada")
df_t

,Especie,Día,Cantidad_Tratada
0,E1,15,575
1,E1,205,152
2,E1,207,450
3,E1,208,1
4,E2,15,1085
...,...,...,...
350,E10,206,9
351,E10,207,9
352,E10,208,9
353,E10,209,4


In [ ]:
df_x = extraer_resultados_variable(x, nombre_columnas=["Especie", "Vivero", "Día"], nombre_valor="Cantidad_Pedida")
df_x

,Especie,Vivero,Día,Cantidad_Pedida
0,E1,V4,12,575
1,E1,V4,202,603
2,E2,V1,194,1
3,E2,V1,200,2
4,E2,V4,12,3425
...,...,...,...,...
84,E10,V1,194,9
85,E10,V1,199,4000
86,E10,V1,200,3998
87,E10,V1,203,9


In [ ]:
df_x.Día.nunique()

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_x)

In [ ]:
df_x.Cantidad_Pedida.sum()

np.int64(180593)

In [ ]:
df_almacen = extraer_resultados_variable(a, nombre_columnas=["Especie", "Día"], nombre_valor="Cantidad_Almacen")
df_almacen

,Especie,Día,Cantidad_Almacen
0,E1,12,575
1,E1,13,575
2,E1,14,575
3,E1,202,603
4,E1,203,603
...,...,...,...
730,E10,206,102377
731,E10,207,102368
732,E10,208,102359
733,E10,209,102359


In [ ]:
df_pl = extraer_resultados_variable(pl, nombre_columnas=["Especie", "Poligono","Dia","Dia de llegada"], nombre_valor="Cantidad_Plantada")
df_pl

,Especie,Poligono,Dia,Dia de llegada,Cantidad_Plantada
0,E1,P1,207,202,90
1,E1,P3,15,12,90
2,E1,P2,207,202,90
3,E1,P12,15,12,90
4,E1,P11,15,12,35
...,...,...,...,...,...
529,E10,P27,51,46,9
530,E10,P27,129,124,9
531,E10,P27,131,124,9
532,E10,P27,170,165,9


In [ ]:
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=df_pl)

In [ ]:
df_pl.Cantidad_Plantada.sum()

np.int64(26689)

In [ ]:
df_pl = extraer_resultados_variable(pi, nombre_columnas=["VIvero","Dia"], nombre_valor="Pedidos")
df_pl

,VIvero,Dia,Pedidos
0,V1,5,1
1,V1,12,1
2,V1,19,1
3,V1,26,1
4,V1,33,1
5,V1,40,1
6,V1,46,1
7,V1,51,1
8,V1,55,1
9,V1,60,1
